In [ ]:
public class Deposito {
    private int items = 0;
    private final int capacidade = 10;

    public synchronized int retirar() {
        if (items > 0) {
            items--;
            System.out.println("Caixa retirada: Sobram " + items + " caixas");
            return 1;
        }
        return 0;
    }
    
    public synchronized int retirar_guarded_suspension() throws InterruptedException {
        while (items == 0) {
            wait();
        }
        items--;
        System.out.println("Caixa retirada: Sobram " + items + " caixas");
        return 1;
    }
    
    public synchronized int retirar_timed_wait(long milliseconds) throws InterruptedException {
        long endTime = System.currentTimeMillis() + milliseconds;
        while (items == 0 && System.currentTimeMillis() < endTime) {
            wait(milliseconds);
        }
        if (items > 0) {
            items--;
            System.out.println("Caixa retirada: Sobram " + items + " caixas");
            return 1;
        }
        return 0;
    }

    public synchronized int colocar() {
        if (items < capacidade) {
            items++;
            System.out.println("Caixa armazenada: Passaram a ser " + items + " caixas");
            return 1;
        }
        return 0;
    }

    public static void main(String[] args) {
        Deposito dep = new Deposito();
        
        Produtor p = new Produtor(dep, 2);
        Consumidor c1 = new Consumidor(dep, 1, 1); // Utiliza o método retirar()
        Consumidor c2 = new Consumidor(dep, 1, 2); // Utiliza o método retirar_guarded_suspension()
        Consumidor c3 = new Consumidor(dep, 1, 3); // Utiliza o método retirar_timed_wait()
    
        Thread produtorThread = new Thread(p);
        Thread consumidorThread1 = new Thread(c1);
        Thread consumidorThread2 = new Thread(c2);
        Thread consumidorThread3 = new Thread(c3);

        produtorThread.start();
        consumidorThread1.start();
        consumidorThread2.start();
        consumidorThread3.start();

        try {
            produtorThread.join();
            consumidorThread1.join();
            consumidorThread2.join();
            consumidorThread3.join();
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        System.out.println("Execução do main da classe Deposito terminada");
    }
}

In [ ]:
class Produtor implements Runnable {
    private Deposito dep;
    private int tempoProducao;

    public Produtor(Deposito dep, int tempoProducao) {
        this.dep = dep;
        this.tempoProducao = tempoProducao;
    }

    public void run() {
        try {
            while (true) {
                dep.colocar();
                Thread.sleep(tempoProducao * 1000);
            }
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }
}

In [ ]:
class Consumidor implements Runnable {
    private Deposito dep;
    private int tempoConsumo;
    private int metodoAcesso;

    public Consumidor(Deposito dep, int tempoConsumo, int metodoAcesso) {
        this.dep = dep;
        this.tempoConsumo = tempoConsumo;
        this.metodoAcesso = metodoAcesso;
    }

    public void run() {
        try {
            while (true) {
                switch (metodoAcesso) {
                    case 1:
                        dep.retirar();
                        break;
                    case 2:
                        dep.retirar_guarded_suspension();
                        break;
                    case 3:
                        dep.retirar_timed_wait(5000);
                        break;
                }
                Thread.sleep(tempoConsumo * 1000);
            }
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    }
}